In [1]:
reload = False
# Import des bibliothèques pour analyse
import folium
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.model_selection import train_test_split,GridSearchCV,cross_validate
from sklearn.linear_model import Ridge, LassoCV, Lasso,LinearRegression
from sklearn.ensemble import RandomForestClassifier

import Bibliotheques.bikebiblio as bike
import Bibliotheques.ReadAndCleanDataBike as dataCleaning
if reload == True :
    df = dataCleaning.readAndClean('Datasets/Bike P(Y)TH - PARIS.csv',';')
else:
    df = dataCleaning.LoadSaved()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 594554 entries, 0 to 594553
Data columns (total 29 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Unnamed: 0         594554 non-null  int64  
 1   idcompteur         594554 non-null  object 
 2   nomcompteur        594554 non-null  object 
 3   idsite             594554 non-null  float64
 4   nomsite            594554 non-null  object 
 5   comptagehoraire    594554 non-null  float64
 6   dateheurecomptage  594554 non-null  object 
 7   dateinstall        594554 non-null  object 
 8   photo              594554 non-null  object 
 9   coord              594554 non-null  object 
 10  date               594554 non-null  object 
 11  datemois           594554 non-null  object 
 12  year               594554 non-null  float64
 13  day                594554 non-null  float64
 14  weekday            594554 non-null  float64
 15  weekofyear         594554 non-null  int64  
 16  mo

In [ ]:
# Affichage de la description du dataframe df
# Création d'un DataFrame avec les variables descriptives uniquement
df_descritive = df.drop(columns=["nomcompteur", "idsite",'nomsite','dateinstall','photo','coord','year','day','weekofyear','lat','long'
                                ,'arrondissement','jourdelasemaine','mois'])
df_descritive.info()

In [ ]:
## RELATION ENTRE VARIABLES NUMERIQUES  "idsite" vs "comptagehoraire"
from scipy.stats import pearsonr
pd.DataFrame(pearsonr(df_descritive['numidcompteur'],df_descritive['comptagehoraire'] ), index=['coef_pearson','p_value'],columns=['Result'] )
# ces 2 variables sont décorelées

In [ ]:
# Affichage de la heat map des corrélations de toutes les variables qualitatives de df
df_descritive.info()
dfcor = df_descritive.corr()
fig, ax = plt.subplots(figsize=(12,12))
sns.heatmap(dfcor, annot= True, ax= ax, cmap="plasma");

In [ ]:
### RELATION ENTRE VARIABLES NUMERIQUES ET CATEGORIELLES : "idsite vs(Coord, nomsite,photo,nomcompteur)"###
    
import statsmodels.api
result = statsmodels.formula.api.ols('idsite ~ coord', data = df).fit()
table1 = statsmodels.api.stats.anova_lm(result)
display(table1)
# le site  fortement correlé aux coordonnées géo.#


result = statsmodels.formula.api.ols('idsite  ~ nomsite', data = df).fit()
table2 = statsmodels.api.stats.anova_lm(result)
display(table2)
#le site  fortement correlé   au nom du lieu.#


result = statsmodels.formula.api.ols('idsite  ~ photo', data = df).fit()
table3 = statsmodels.api.stats.anova_lm(result)
display(table3)
#le site  fortement correlé à la photo du site.#

result = statsmodels.formula.api.ols('idsite  ~ nomcompteur', data = df).fit()
table4 = statsmodels.api.stats.anova_lm(result)
display(table4)
#le site  fortement correlé du  nom du COMPTEUR.#

DATAVIZ

In [ ]:
# Répartition du comptage par mois
functions_to_apply = {
    'comptagehoraire' : lambda comptage: comptage.mean(),
    'month' : lambda month: month.mean()
    }

dfparmois = df.groupby('mois',as_index = False).agg(functions_to_apply)
dfparmois = dfparmois.sort_values(by='month', ascending=True)
dfparmois=dfparmois.reset_index()

fig, ax = plt.subplots(figsize=(10,10))
sns.barplot(x=dfparmois.mois, y=dfparmois["comptagehoraire"], ax=ax)
ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="right");
ax.set_title("Répartition comptage par mois")
ax.set_ylabel('Nombre moyen de passage par heure')


plt.show()
display(dfparmois.drop(columns=['month','index']))

In [ ]:
# Répartition du comptage par mois
plt.figure(figsize=(12,12))
sns.lmplot(x="month", y="comptagehoraire", data=dfparmois, order=3)
plt.title('Répartition comptage par mois');
plt.xlabel("Mois")
plt.ylabel("Nombre moyen de passage par heure")

Au vue de ces 2 graphiques, il y a une tendance à privilégier ce mode de transport en fin d'année. 
En effet, les mois de novembre et décembre comptabilise le plus de cycliste.
Il serait intéressant de voir la corrélation entre cette tendance et des évènements sur la circulation qui serait arrivés sur la période ?

In [ ]:
# Répartition par jour 
functions_to_apply = {
    'comptagehoraire' : lambda comptage: comptage.mean(),
    'weekday' : lambda weekday: weekday.mean()
    }

dfparjour = df.groupby('jourdelasemaine',as_index = False).agg(functions_to_apply)
dfparjour = dfparjour.sort_values(by='weekday', ascending=True)
dfparjour.reset_index()

display(dfparjour.drop(columns=['weekday']))
fig, ax = plt.subplots(figsize=(10,10))
sns.barplot(x=dfparjour.jourdelasemaine, y=dfparjour["comptagehoraire"], ax=ax)
ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="right");
plt.xlabel("")
plt.ylabel("Nombre moyen de passage par heure")

plt.show()

On notera une baisse du trafic le weekend, ce mode de transport semblerait plus destibné à aller au travail que pour se promener.

In [ ]:
functions_to_apply = {
    'comptagehoraire' : lambda comptage: comptage.mean()
    }

dfparmoisannee = df.sort_values(by='datemois', ascending=True).groupby('datemois',as_index = False).agg(functions_to_apply)
#dfparmoisannee = dfparmoisannee.sort_values(by='datemois', ascending=True)
dfparmoisannee.reset_index()

fig, ax = plt.subplots(figsize=(10,10))
sns.barplot(x=dfparmoisannee.datemois, y=dfparmoisannee["comptagehoraire"], ax=ax)
ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="right");
ax.set_title("Répartition comptage par mois")
ax.set_ylabel('Nombre moyen de passage par heure')
ax.set_xlabel('')


plt.show()

La fin d'année 2019 rencontre la plus grosse affluence de cycliste, il y a suûrement un parallèle avec les grèves intervenus fin 2019 et le ras le bol des metro user.

In [ ]:
functions_to_apply = {
    'comptagehoraire' : lambda comptage: comptage.mean(),
    'lat' : lambda lat:lat.max(),
    'long' : lambda long:long.max()
}

df_grouped = df.groupby('idcompteur').agg(functions_to_apply)

df_grouped = df_grouped.dropna()
df_grouped.describe()


def getColor(nb_passage,nb_max):
    tier_max = nb_max/3
    if(tier_max > nb_passage):
        return '#ee114d'
    if( tier_max < nb_passage < (tier_max*2)):        
        return '#eed311'
    if((tier_max*2) < nb_passage  ):        
        return '#28c0e2'
    
nb_max = df_grouped.comptagehoraire.max()

map = folium.Map(width=550,height=350,location = [48.856578, 2.351828], zoom_start = 11, tiles='OpenStreetMap')
for ligne in df_grouped.T:
    t_radius = df_grouped.T[ligne].comptagehoraire/nb_max*30
    folium.CircleMarker(
       location = (df_grouped.T[ligne].lat, df_grouped.T[ligne].long),
       radius = t_radius,
       color = getColor(df_grouped.T[ligne].comptagehoraire,nb_max),
       fill = True,
       fill_color = getColor(df_grouped.T[ligne].comptagehoraire,nb_max),
       popup = str(df_grouped.T[ligne].comptagehoraire)
    ).add_to(map)

folium.Choropleth(
       geo_data='datas/departement-75-paris.geojson',fill_opacity=0.1
                     
    ).add_to(map)
display(map)     



Les visualisations suivantes permettent de voir le traffic par arrondissement.
Ici sont visibles une visualisation à 16h, une à 1h du matin. 
Nous observons également la différence de fréquentation les weekend et en semaine

In [ ]:

display('A 16 heures')
mask =  (df['hour']==16)   
df_grouped,map = bike.GetMapParArrondissement(df,mask,True) 
display(map)

display('A 1 heures')
mask =  (df['hour']==1)
df_grouped,map = bike.GetMapParArrondissement(df,mask,True) 
display(map)

display('En semaine')
mask =  (df['weekday']!=6) | (df['weekday']!=5)
df_grouped,map = bike.GetMapParArrondissement(df,mask,True) 
display(map)

display('Weekend')
mask =  (df['weekday']==6)| (df['weekday']==5)
df_grouped,map = bike.GetMapParArrondissement(df,mask,True) 
display(map)



In [ ]:

display('Jour de grève')
mask =  (df['jourdegreve']==1)   
df_grouped,map = bike.GetMapParArrondissement(df,mask,True) 
display(map)

display('Jours fériés')
mask =  (df['holiday']==1)
df_grouped,map = bike.GetMapParArrondissement(df,mask,True) 
display(map)

display('Vacances scolaires')
mask =  (df['vacances_zone_c']==1)
df_grouped,map = bike.GetMapParArrondissement(df,mask,True) 
display(map)

display('Les autres')
mask =   (df['holiday']==0) & (df['jourdegreve']==0) & (df['vacances_zone_c']==0)
df_grouped,map = bike.GetMapParArrondissement(df,mask,True) 
display(map)



In [ ]:
for temps in df['meteo_cat'].unique():
    print(temps)
    mask = df.meteo_cat == temps
    df_grouped,map = bike.GetMapParArrondissement(df,mask,True) 
    display(map)

In [ ]:
functions_to_apply = {
    'comptagehoraire' : lambda comptage: comptage.mean()
    }
fig, (ax1, ax2,ax3) = plt.subplots(1, 3, figsize=(20,10))
dfgrev = df.groupby('jourdegreve',as_index = False).agg(functions_to_apply)
    
sns.barplot(x=dfgrev.jourdegreve, y=dfgrev["comptagehoraire"], ax=ax1)
ax1.set_xticklabels(ax1.get_xticklabels(), rotation=40, ha="right");
ax1.set_title("Jour de grève")
ax1.set_ylabel('Nombre moyen de passage par heure')
ax1.set_xlabel('')

functions_to_apply = {
    'comptagehoraire' : lambda comptage: comptage.mean()
    }
dfvacances = df.groupby('vacances_zone_c',as_index = False).agg(functions_to_apply)    
sns.barplot(x=dfvacances.vacances_zone_c, y=dfvacances["comptagehoraire"], ax=ax2)
ax2.set_xticklabels(ax2.get_xticklabels(), rotation=40, ha="right")
ax2.set_title("Vacances Scolaire")
ax2.set_ylabel('Nombre moyen de passage par heure')
ax2.set_xlabel('')

functions_to_apply = {
    'comptagehoraire' : lambda comptage: comptage.mean()
    }

dfferie = df.groupby('holiday',as_index = False).agg(functions_to_apply)
    
sns.barplot(x=dfferie.holiday, y=dfferie["comptagehoraire"], ax=ax3)
ax3.set_xticklabels(ax3.get_xticklabels(), rotation=40, ha="right")
ax3.set_title("Jours Fériés")
ax3.set_ylabel('Nombre moyen de passage par heure')
ax3.set_xlabel('')


plt.show()

Ici nous pouvons voir les spots les plus actifs en terme de comptage de cyclistes.

In [ ]:
functions_to_apply = {
    'comptagehoraire' : lambda comptage: comptage.mean(),
    'nomsite' : lambda comptage: comptage.max(),
    'arrondissement' : lambda comptage: comptage.max()
    }

dfparcompteur = df.groupby(['idsite'],as_index = True).agg(functions_to_apply)
dfparcompteur = dfparcompteur.sort_values(by='comptagehoraire', ascending=True)
dfparcompteur.reset_index()

df_head = dfparcompteur.head()
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
sns.barplot(x=df_head.nomsite, y=df_head["comptagehoraire"], ax=ax1)
ax1.set_xticklabels(ax1.get_xticklabels(), rotation=40, ha="right");
ax1.set_title("Sites ayant le moins de passages")
ax1.set_ylabel('Nombre moyen de passage par heure')
ax1.set_xlabel('')


df_tail = dfparcompteur.tail()
sns.barplot(x=df_tail.nomsite, y=df_tail["comptagehoraire"], ax=ax2)
ax2.set_xticklabels(ax2.get_xticklabels(), rotation=40, ha="right");
ax2.set_title("Sites ayant le plus de passages")
ax2.set_ylabel('Nombre moyen de passage par heure')
ax2.set_xlabel('')


plt.show()


Les spots qui comptabilisent le plus de cycliste sont dans le 10ème LAFAYETTE, dans le 19ème quai de la Marne et dans le 11ème bvd Voltaire. A quoi est dû cette concentration ? Est ce les aménagements en pistes cyclables ? 
Est des lieux plus attractifs pour les sorties, le travail ? Il serait bon de les comparés avec les points les moins attractifs pour faire une évaluation des différences et comprendre la tendance.

Ainsi, nous avons pu voir plusieurs tendances sur les périodes qui comptabilisent le plus de comptage, les lieux les plus chaud ou les disparités semaine/weekend.
Il serait bon de croiser toutes ces données et voir si un modèle de prédiction pouvait nous aider à répondre à notre problématique et permettre d'anticiper ces mouvements.

In [ ]:
g = sns.PairGrid(df_descritive, hue= 'numidcompteur')
g.map(plt.scatter, alpha=0.6)
g.add_legend()

In [ ]:
# Modélisation d'une régression linéaire multiple 
plt.figure(figsize= (10,8))
plt.scatter(df['hour'], df.comptagehoraire, color = 'darkblue');


In [ ]:
target = df['comptagehoraire']
data = df[['hour']]

slr = LinearRegression()
slr.fit(data, target)

In [ ]:
target = df_descritive['comptagehoraire']
data = df_descritive[['numidcompteur','weekday','month','hour','jourdegreve','holiday','vacances_zone_c']]
df_unique = df_descritive[df['arrondissement'] == 19]
print(df_unique.info())
target = df_unique['comptagehoraire']
data = df_unique[['weekday','month','hour','jourdegreve','holiday','vacances_zone_c']]
slr = LinearRegression()
slr.fit(data, target)

In [ ]:
print(slr.intercept_)
print(slr.coef_)

In [ ]:
cross_validate(slr, data, target, return_train_score=True, cv=4)

In [ ]:
cross_validate(slr, data, target, return_train_score=True, cv=4)['test_score'].mean()

In [ ]:
pred_target = slr.predict(data)
residus = pred_target - target
residus.describe()

In [ ]:
from sklearn.feature_selection import f_regression
print('F-score:', f_regression(data, target)[0], 'p-value :', f_regression(data, target)[1])

In [ ]:
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())
rmse(pred_target, target)

# Modélissation Lasso et RandomForest
Dans un premier temps, il a été décidé d'utiliser ces modèles sur l'ensemble du jeux de données. Cependant, les résultats n'étaient pas bon, et le temps de traitement était trop élevé.
Nous avons décidé de diviser notre dataset par arrondissement pour voir si par arrondissement les modèles était plus fiable.Le temps de traitement est ainsi devenu viable.

Paramètres test Lasso
Pour le modèle Lasso, nous avons utilisé les valeurs d'alpha suivantes : [10,1,0.1,0.001,0.0005]
Pour le modèle Random Forest, nous avons utilisé les paramètres de tests suivants:
param_grid_rf_  = [{'n_estimators': [10, 50, 100, 1000],
                         'min_samples_leaf': [1, 3, 5],
                         'max_features': ['sqrt', 'log2']}]
                         
Voici pour chaque arrondissement les résultats des tests sur les modèles Lasso et Forest; ainsi que pour chacun des deux les arrondissements ayant obtenus les meilleurs scores.


In [ ]:
def testLasso(X_train, X_test, y_train, y_test):
    vec = [10,1,0.1,0.001,0.0005]
    lasso_reg = LassoCV(alphas= vec)
    lasso_reg.fit(X_train, y_train) 
    lasso_r = Lasso(lasso_reg.alpha_)
    lasso_r.fit(X_train, y_train)
    return lasso_r.score(X_test, y_test)

In [ ]:
def testRandomForest(X_train, X_test, y_train, y_test):
    clf_rf  = RandomForestClassifier()
    param_grid_rf  = [{'n_estimators': [10, 100],
                         'min_samples_leaf': [1],
                         'max_features': ['sqrt']}]
    gridcv = GridSearchCV(clf_rf,
                           param_grid=param_grid_rf,
                           scoring='accuracy',
                           cv=2) 

    gridcv.fit(X_train, y_train)
    return gridcv.score(X_test, y_test)

In [ ]:

resultatsTest = pd.DataFrame( index = df.arrondissement.unique(), columns = ['arrondissement','scoreLasso','scoreForest'])
resultatsTest.sort_index()
for arrondissement in resultatsTest.T:   
    print(arrondissement)
    df_unique = df_descritive[df['arrondissement'] == arrondissement]
    target = df_unique['comptagehoraire']
    feats = df_unique[['weekday','month','hour','jourdegreve','holiday','vacances_zone_c']]
    encoder=OneHotEncoder()
    features_enc=encoder.fit_transform(feats)
    X_train, X_test, y_train, y_test = train_test_split(feats, target, test_size=0.2)
    
    resultLasso = testLasso(X_train, X_test, y_train, y_test)
    resultForest = testRandomForest(X_train, X_test, y_train, y_test)
    resultatsTest.loc[arrondissement]= [arrondissement,resultLasso, resultForest ]
    

In [ ]:
resultatsTest = resultatsTest.sort_index()
display(resultatsTest)
resultLasso=resultatsTest[['arrondissement','scoreLasso']]
resultForest=resultatsTest[['arrondissement','scoreForest']]

display(resultLasso.sort_values(by='scoreLasso',ascending=False).head())
display(resultForest.sort_values(by='scoreForest',ascending=False).head())